LRU 缓存淘汰算法就是一种常用策略。LRU 的全称是 Least Recently Used，也就是说我们认为最近使用过的数据应该是是「有用的」，很久都没用过的数据应该是无用的，内存满了就优先删那些很久没用过的数据。


请你设计并实现一个满足  LRU (最近最少使用) 缓存 约束的数据结构。<br>

实现 LRUCache 类：<br>
LRUCache(int capacity) 以 正整数 作为容量 capacity 初始化 LRU 缓存<br>
int get(int key) 如果关键字 key 存在于缓存中，则返回关键字的值，否则返回 -1 。<br>
void put(int key, int value) 如果关键字 key 已经存在，则变更其数据值 value ；<br>
如果不存在，则向缓存中插入该组 key-value 。<br>
如果插入操作导致关键字数量超过 capacity ，则应该 逐出 最久未使用的关键字。<br>
函数 get 和 put 必须以 O(1) 的平均时间复杂度运行。<br>

 

示例：

输入<br>
["LRUCache", "put", "put", "get", "put", "get", "put", "get", "get", "get"]<br>
[[2], [1, 1], [2, 2], [1], [3, 3], [2], [4, 4], [1], [3], [4]]<br>
输出<br>
[null, null, null, 1, null, -1, null, -1, 3, 4]<br>

解释

LRUCache lRUCache = new LRUCache(2); <br>
lRUCache.put(1, 1); // 缓存是 {1=1}<br>
lRUCache.put(2, 2); // 缓存是 {1=1, 2=2}<br>
lRUCache.get(1);    // 返回 1<br>
lRUCache.put(3, 3); // 该操作会使得关键字 2 作废，缓存是 {1=1, 3=3}<br>
lRUCache.get(2);    // 返回 -1 (未找到)<br>
lRUCache.put(4, 4); // 该操作会使得关键字 1 作废，缓存是 {4=4, 3=3}<br>
lRUCache.get(1);    // 返回 -1 (未找到)<br>
lRUCache.get(3);    // 返回 3<br>
lRUCache.get(4);    // 返回 4<br>
 

提示：

1 <= capacity <= 3000<br>
0 <= key <= 10000<br>
0 <= value <= 105<br>
最多调用 2 * 105 次 get 和 put<br>

In [ ]:
class DLinkedNode:
    # 功能: 定义一个双向链表节点类型，用于LRUCache中存储键值对。
    # 原因: 需要支持在O(1)时间内进行插入和删除操作，双向链表是最佳选择。

    def __init__(self, key=0, value=0):
        # 功能: 初始化节点对象并设置键、值以及前后指针。
        # 原因: 节点必须能保存数据并能被快速双向访问。

        self.key = key
        # 功能: 存储键，以便在淘汰节点时能在哈希表中找到对应项删除。
        # 原因: LRU缓存需要从节点反查哈希表key，不能只存value。

        self.value = value
        # 功能: 存储缓存的实际值。
        # 原因: get(key)最终要返回的就是这个值。

        self.prev = None
        # 功能: 指向前驱节点。
        # 原因: 允许在O(1)时间删除节点时更新相邻指针。

        self.next = None
        # 功能: 指向后继节点。
        # 原因: 允许在O(1)时间插入或移动节点时连接结构。


class LRUCache:
    # 功能: 实现一个固定容量的最近最少使用缓存（Least Recently Used）。
    # 原因: 提供O(1)时间复杂度的访问与更新，并能自动淘汰最久未使用的数据。

    def __init__(self, capacity: int):
        # 功能: 初始化LRU缓存，设置容量、哈希表和双向链表的伪头尾。
        # 原因: 初始化这些结构是为了实现高效的访问和更新操作。

        self.cache = dict()
        # 功能: 用字典存储key到节点的映射。
        # 原因: 哈希表能让get和put操作的查找为O(1)。

        self.head = DLinkedNode()
        # 功能: 创建伪头节点(dummy head)。
        # 原因: 简化插入逻辑，避免处理空链表或单节点的特殊情况。

        self.tail = DLinkedNode()
        # 功能: 创建伪尾节点(dummy tail)。
        # 原因: 简化删除尾节点时的逻辑，保持统一的结构。

        self.head.next = self.tail
        # 功能: 让伪头的next指向伪尾。
        # 原因: 初始化时构造一个空链表结构，head <-> tail。

        self.tail.prev = self.head
        # 功能: 让伪尾的prev指向伪头。
        # 原因: 形成双向链接结构，保证链表完整。

        self.capacity = capacity
        # 功能: 保存最大缓存容量。
        # 原因: 后续put时用于判断是否需要淘汰老节点。

        self.size = 0
        # 功能: 当前缓存中已存储的节点数量。
        # 原因: 快速判断是否超过容量，无需遍历链表。


    def get(self, key: int) -> int:
        # 功能: 读取指定key的值，并将该节点移动到链表头部（表示最近使用）。
        # 原因: LRU策略要求每次访问都更新使用顺序。

        if key not in self.cache:
            # 功能: 判断key是否存在于缓存。
            # 原因: 如果不存在，说明缓存未命中，应返回-1。
            return -1
            # 功能: 返回-1表示缓存未命中。
            # 原因: 通常LRU接口标准要求未命中返回-1。

        node = self.cache[key]
        # 功能: 从哈希表中O(1)定位到节点。
        # 原因: 直接查字典避免链表遍历，保证高效性。

        self.moveToHead(node)
        # 功能: 将该节点移动到链表头部。
        # 原因: 最近被访问的节点应标记为“最新使用”，防止被淘汰。

        return node.value
        # 功能: 返回节点的值。
        # 原因: get的最终目标是获取缓存数据。


    def put(self, key: int, value: int) -> None:
        # 功能: 插入或更新key-value对。
        # 原因: LRUCache的主要写操作，需要在插入时更新使用顺序并控制容量。

        if key not in self.cache:
            # 功能: 判断key是否为新项。
            # 原因: 插入新节点和更新旧节点的逻辑不同。

            node = DLinkedNode(key, value)
            # 功能: 创建新节点。
            # 原因: 缓存中不存在该key，需新建节点。

            self.cache[key] = node
            # 功能: 在哈希表中记录映射。
            # 原因: 通过哈希表快速定位节点，实现O(1)查找。

            self.addToHead(node)
            # 功能: 将新节点加入链表头部。
            # 原因: 新插入的节点应视为最近使用的节点。

            self.size += 1
            # 功能: 更新缓存当前大小。
            # 原因: 记录缓存中元素数量以判断是否超出容量。

            if self.size > self.capacity:
                # 功能: 检查是否超出最大容量。
                # 原因: 若超出，需要淘汰最久未使用节点。

                removed = self.removeTail()
                # 功能: 移除链表尾部的节点。
                # 原因: 尾部是最久未使用的节点，符合LRU淘汰策略。

                self.cache.pop(removed.key)
                # 功能: 从哈希表中删除该节点对应的key。
                # 原因: 保持哈希表和链表状态一致，防止悬挂引用。

                self.size -= 1
                # 功能: 缩减计数。
                # 原因: 保持缓存大小与实际数据同步。

        else:
            # 功能: 处理已存在的key。
            # 原因: 若key已存在，只需更新值并刷新使用顺序。

            node = self.cache[key]
            # 功能: 获取已有节点。
            # 原因: 不需新建节点，只需修改其值。

            node.value = value
            # 功能: 更新节点的值。
            # 原因: 保证缓存中的值始终最新。

            self.moveToHead(node)
            # 功能: 将节点移到链表头部。
            # 原因: 表示该节点刚被使用，更新其LRU状态。


    def addToHead(self, node):
        # 功能: 将节点插入链表头部。
        # 原因: LRU逻辑要求新节点或被访问节点都放在最前面。

        node.prev = self.head
        # 功能: 设置节点前驱为伪头。
        # 原因: 这是插入到头部所需的指针结构。

        node.next = self.head.next
        # 功能: 设置节点后继为原头部第一个节点。
        # 原因: 保持原链表顺序连接。

        self.head.next.prev = node
        # 功能: 更新原头节点的前驱指向新节点。
        # 原因: 保证双向链表结构完整。

        self.head.next = node
        # 功能: 让伪头的next指向新节点。
        # 原因: 完成头部插入操作。


    def removeNode(self, node):
        # 功能: 将节点从链表中移除。
        # 原因: 删除节点时需更新相邻指针，保证O(1)复杂度。

        node.prev.next = node.next
        # 功能: 前驱节点的next跳过当前节点。
        # 原因: 断开当前节点的前向连接。

        node.next.prev = node.prev
        # 功能: 后继节点的prev跳过当前节点。
        # 原因: 断开当前节点的后向连接。


    def moveToHead(self, node):
        # 功能: 将节点移动到链表头部。
        # 原因: 最近使用的节点要置于最前面以维持LRU顺序。

        self.removeNode(node)
        # 功能: 从当前位置删除节点。
        # 原因: 防止重复引用，准备重新插入。

        self.addToHead(node)
        # 功能: 重新插入到链表头部。
        # 原因: 表示该节点刚被访问，是最新使用的。


    def removeTail(self):
        # 功能: 删除并返回链表尾部节点。
        # 原因: 尾部是最久未使用的节点，LRU淘汰时应优先移除。

        node = self.tail.prev
        # 功能: 获取伪尾前一个节点。
        # 原因: 这是最后一个真实节点，也是最久未使用的。

        self.removeNode(node)
        # 功能: 从链表中移除该节点。
        # 原因: 实现O(1)的删除操作。

        return node
        # 功能: 返回被删除节点。
        # 原因: 方便上层从哈希表中同步移除。
